In [1]:
import os.path
from xml.dom import minidom

In [2]:
# Output directory
out_dir = '../Antelope/Annotation'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Parse the XML file
file = minidom.parse('../Antelope/Annotation/annotations.xml')

# Extract all image elements
images = file.getElementsByTagName('image')

# Iterate over each image element
for image in images:
    width = int(image.getAttribute('width'))
    height = int(image.getAttribute('height'))
    name = image.getAttribute('name')

    # Get the bounding box details
    bbox = image.getElementsByTagName('box')[0]
    xtl = int(float(bbox.getAttribute('xtl')))
    ytl = int(float(bbox.getAttribute('ytl')))
    xbr = int(float(bbox.getAttribute('xbr')))
    ybr = int(float(bbox.getAttribute('ybr')))
    w = xbr - xtl
    h = ybr - ytl

    # Open a label file for the image
    label_file = open(os.path.join(out_dir, name[:-4] + '.txt'), 'w')

    # Process the points (keypoints) for the image
    points_elements = image.getElementsByTagName('points')
    for points_elem in points_elements:
        points_str = points_elem.getAttribute('points')
        
        # Split the points by ';' and further split each point by ',' to extract x, y coordinates
        points = points_str.split(';')
        points_ = []
        for p in points:
            p = p.split(',')
            p1, p2 = p
            points_.append([int(float(p1)), int(float(p2))])

        # Write the YOLO format for bounding box (class 0 for quadruped)
        label_file.write(f"0 {(xtl + (w / 2)) / width} {(ytl + (h / 2)) / height} {w / width} {h / height} ")

        # Write normalized keypoints (if any)
        for p_ in points_:
            label_file.write(f"{p_[0] / width} {p_[1] / height} ")

        # End the line in the label file
        label_file.write('\n')

    # Close the label file
    label_file.close()